In [1]:
import pandas as pd
import numpy as np
import pickle
from ase import Atoms
from dscribe.descriptors import ACSF
import copy

In [3]:
#Atoms(symbols,positions)

In [2]:
train = pd.read_csv('../Data/train.csv')
test = pd.read_csv('../Data/test.csv')
structures = pd.read_csv('../Data/structures.csv')
train_bonds = pd.read_csv('../Data/train_bonds.csv')
test_bonds = pd.read_csv('../Data/test_bonds.csv')

In [3]:
test_bonds = test_bonds.drop('Unnamed: 0',1)
train_bonds = train_bonds.drop('Unnamed: 0',1)

In [ ]:
train.head(10)

In [ ]:
structures.head()

In [ ]:
train_bonds.head()

In [ ]:
test_bonds.head()

### node information

In [6]:
structures[['C', 'F', 'H', 'N', 'O']] = pd.get_dummies(structures.atom)
structures = structures.sort_values(by=['molecule_name', 'atom_index'])

In [7]:
structures_gb = structures.groupby(['molecule_name'])

In [8]:
structures_dict = {}
for k,v in structures_gb:
    atom_dict = {'positions':v[['x','y','z']].values.tolist(),
                 'symbols':[i[0] for i in v[['atom']].values.tolist()]
                 }
    atom = Atoms(**atom_dict)
    dist = atom.get_all_distances()
    d = dist.shape[0]
    dist = dist[np.tril_indices(d,-1)]
    r_info = np.array([dist.max(),dist.min(),dist.mean(),dist.std()])
    
    acsf = ACSF(species=['C', 'F', 'H', 'N', 'O'],
                rcut=r_info[0]+0.5,
                g2_params=[[1, 1], [1, 2], [1, 3]],
                g4_params=[[1, 1, 1], [1, 2, 1], [1, 1, -1], [1, 2, -1]])
    acsf_np = acsf.create(atom)
    structures_dict[k] = np.concatenate([acsf_np,v[['C', 'F', 'H', 'N', 'O']].values,\
                                         np.tile(r_info,(d,1))],1).astype(np.float32)

In [9]:
with open('../Data/structures_dict_ACSF_3_4.pickle', 'wb') as handle:
    pickle.dump(structures_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

### bond information

In [10]:
assert set(train_bonds.bond_type) == set(test_bonds.bond_type)

In [11]:
bonds = train_bonds.append(test_bonds, ignore_index=True)

In [12]:
del train_bonds,test_bonds

In [13]:
bonds[['1.0CC',
 '1.0CF',
 '1.0CH',
 '1.0CN',
 '1.0CO',
 '1.0HN',
 '1.0HO',
 '1.0NN',
 '1.0NO',
 '1.5CO',
 '2.0CC',
 '2.0CN',
 '2.0CO',
 '2.0NN',
 '2.0NO',
 '3.0CC',
 '3.0CN']] = pd.get_dummies(bonds.bond_type)

In [14]:
bonds.head()

,molecule_name,atom_index_0,atom_index_1,nbond,L2dist,error,bond_type,1.0CC,1.0CF,1.0CH,...,1.0NN,1.0NO,1.5CO,2.0CC,2.0CN,2.0CO,2.0NN,2.0NO,3.0CC,3.0CN
0,dsgdb9nsd_000001,0,1,1.0,1.091953,0,1.0CH,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,dsgdb9nsd_000001,0,2,1.0,1.091952,0,1.0CH,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,dsgdb9nsd_000001,0,3,1.0,1.091946,0,1.0CH,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,dsgdb9nsd_000001,0,4,1.0,1.091948,0,1.0CH,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,dsgdb9nsd_000002,0,1,1.0,1.017190,0,1.0HN,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
bonds_gb = bonds.groupby(['molecule_name'])

In [16]:
bonds_edge_index = {}
bonds_edge_attr = {}
for k,v in bonds_gb:
    bonds_edge_index[k] = np.concatenate([v[['atom_index_0','atom_index_1']].values,\
                                          v[['atom_index_1','atom_index_0']].values]).T
    bonds_edge_attr[k] = np.tile(v[['L2dist', 'error',\
                                   '1.0CC', '1.0CF', '1.0CH', '1.0CN', '1.0CO','1.0HN',\
                                   '1.0HO', '1.0NN', '1.0NO', '1.5CO', '2.0CC', '2.0CN',\
                                   '2.0CO', '2.0NN', '2.0NO', '3.0CC', '3.0CN']].values.astype(np.float32),(2,1))

In [17]:
with open('../Data/bonds_edge_index.pickle', 'wb') as handle:
    pickle.dump(bonds_edge_index, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('../Data/bonds_edge_attr.pickle', 'wb') as handle:
    pickle.dump(bonds_edge_attr, handle, protocol=pickle.HIGHEST_PROTOCOL)    

### coupling information

In [4]:
assert set(train.type) == set(test.type)

In [5]:
test['scalar_coupling_constant'] = np.nan

In [6]:
coupling = train.append(test, ignore_index=True)

In [7]:
coupling.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074


In [8]:
def map_atom_info(df, atom_idx):
    df = pd.merge(df, structures, how = 'left',
                  left_on  = ['molecule_name', f'atom_index_{atom_idx}'],
                  right_on = ['molecule_name',  'atom_index'])
    
    df = df.drop('atom_index', axis=1)
    df = df.rename(columns={'atom': f'atom_{atom_idx}',
                            'x': f'x_{atom_idx}',
                            'y': f'y_{atom_idx}',
                            'z': f'z_{atom_idx}'})
    return df

In [9]:
coupling = map_atom_info(coupling, 0)
coupling = map_atom_info(coupling, 1)

In [12]:
# coupling = coupling.drop(['atom_0', 'C_x', 'F_x',
#                           'H_x', 'N_x', 'O_x', 'atom_1','C_y', 'F_y', 'H_y',
#                           'N_y', 'O_y'], axis=1)

In [13]:
train_p_0 = coupling[['x_0', 'y_0', 'z_0']].values
train_p_1 = coupling[['x_1', 'y_1', 'z_1']].values
coupling['dist'] = np.linalg.norm(train_p_0 - train_p_1, axis=1)

In [14]:
coupling[['1JHC', '1JHN', '2JHC', '2JHH', '2JHN', '3JHC', '3JHH', '3JHN']]=pd.get_dummies(coupling.type)

In [15]:
coupling

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_0,x_0,y_0,z_0,...,z_1,dist,1JHC,1JHN,2JHC,2JHH,2JHN,3JHC,3JHH,3JHN
0,0,dsgdb9nsd_000001,1,0,1JHC,84.80760,H,0.002150,-0.006031,0.001976,...,0.008001,1.091953,1,0,0,0,0,0,0,0
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.25700,H,0.002150,-0.006031,0.001976,...,0.000277,1.783120,0,0,0,1,0,0,0,0
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.25480,H,0.002150,-0.006031,0.001976,...,-0.876644,1.783147,0,0,0,1,0,0,0,0
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.25430,H,0.002150,-0.006031,0.001976,...,0.906397,1.783157,0,0,0,1,0,0,0,0
4,4,dsgdb9nsd_000001,2,0,1JHC,84.80740,H,1.011731,1.463751,0.000277,...,0.008001,1.091952,1,0,0,0,0,0,0,0
5,5,dsgdb9nsd_000001,2,3,2JHH,-11.25410,H,1.011731,1.463751,0.000277,...,-0.876644,1.783158,0,0,0,1,0,0,0,0
6,6,dsgdb9nsd_000001,2,4,2JHH,-11.25480,H,1.011731,1.463751,0.000277,...,0.906397,1.783148,0,0,0,1,0,0,0,0
7,7,dsgdb9nsd_000001,3,0,1JHC,84.80930,H,-0.540815,1.447527,-0.876644,...,0.008001,1.091946,1,0,0,0,0,0,0,0
8,8,dsgdb9nsd_000001,3,4,2JHH,-11.25430,H,-0.540815,1.447527,-0.876644,...,0.906397,1.783148,0,0,0,1,0,0,0,0
9,9,dsgdb9nsd_000001,4,0,1JHC,84.80950,H,-0.523814,1.437933,0.906397,...,0.008001,1.091948,1,0,0,0,0,0,0,0


In [16]:
coupling = coupling.groupby(['molecule_name'])

In [ ]:
coupling_edge_index = {}
coupling_edge_attr = {}
coupling_edge_dist = {}
coupling_y = {}
coupling_id = {}
for k,v in coupling:
    coupling_edge_index[k] = v[['atom_index_0','atom_index_1']].values.T
    coupling_edge_attr[k] = v[['1JHC','1JHN','2JHC','2JHH','2JHN','3JHC','3JHH','3JHN']].values.astype(np.float32)
    coupling_edge_dist[k] = v[['dist']].values.astype(np.float32)
    if not np.any(np.isnan(v.scalar_coupling_constant.values)):
        coupling_y[k] = v.scalar_coupling_constant.values.astype(np.float32)

In [ ]:
with open('../Data/coupling_edge_index.pickle', 'wb') as handle:
    pickle.dump(coupling_edge_index, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('../Data/coupling_edge_attr.pickle', 'wb') as handle:
    pickle.dump(coupling_edge_attr, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('../Data/coupling_edge_dist.pickle', 'wb') as handle:
    pickle.dump(coupling_edge_dist, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('../Data/coupling_y.pickle', 'wb') as handle:
    pickle.dump(coupling_y, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [17]:
coupling_id = {}
for k,v in coupling:
    coupling_id[k] = v.id.values

In [18]:
with open('../Data/coupling_id.pickle', 'wb') as handle:
    pickle.dump(coupling_id, handle, protocol=pickle.HIGHEST_PROTOCOL)  

In [77]:
train_mol = np.unique(train.molecule_name)
test_mol = np.unique(test.molecule_name)

In [78]:
train_mol = np.random.permutation(train_mol)

In [79]:
train_mol2 = train_mol[:70000]
val_mol = train_mol[70000:]
train_mol = train_mol2

In [91]:
def create_data(mols,IsTrain):
    type_list = [[] for _ in range(8)]
    tot_list = []
    if not IsTrain:
        test_id_type_list = [[] for _ in range(8)]
        test_id_list = []
        
    for m in mols:
        if IsTrain:
            dict_ = {'x':structures_dict[m],'edge_index':bonds_edge_index[m],\
                               'edge_attr':bonds_edge_attr[m],'y':coupling_y[m],\
                               'edge_index3':coupling_edge_index[m],'edge_attr3':coupling_edge_attr[m],\
                               'edge_attr4':coupling_edge_dist[m]}
            tot_list.append(copy.deepcopy(dict_))
                        
            temp = dict_['edge_attr3'].argmax(1)
            for i in np.nonzero(dict_['edge_attr3'].sum(0))[0]:
                dict_['type_attr'] = (temp==i).astype(np.uint8)
                type_list[i].append(copy.deepcopy(dict_))
        else:
            dict_ = {'x':structures_dict[m],'edge_index':bonds_edge_index[m],\
                       'edge_attr':bonds_edge_attr[m],\
                       'edge_index3':coupling_edge_index[m],'edge_attr3':coupling_edge_attr[m],\
                       'edge_attr4':coupling_edge_dist[m]}
            tot_list.append(copy.deepcopy(dict_))
            test_id_list.append(coupling_id[m])
            
            temp = dict_['edge_attr3'].argmax(1)
            for i in np.nonzero(dict_['edge_attr3'].sum(0))[0]:
                dict_['type_attr'] = (temp==i).astype(np.uint8)
                type_list[i].append(copy.deepcopy(dict_))
                test_id_type_list[i].append(coupling_id[m][temp==i])
    
    if IsTrain:
        return tot_list,type_list 
    else:
        return tot_list,type_list,np.concatenate(test_id_list),[np.concatenate(type_i) for type_i in test_id_type_list]

In [98]:
tot_list_train,type_list_train = create_data(train_mol,True)
tot_list_val,type_list_val = create_data(val_mol,True)
tot_list_test,type_list_test,test_id,test_id_type = create_data(test_mol,False)

In [99]:
with open('../Data/train_data_ACSF.pickle', 'wb') as handle:
    pickle.dump(tot_list_train, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('../Data/val_data_ACSF.pickle', 'wb') as handle:
    pickle.dump(tot_list_val, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('../Data/test_data_ACSF.pickle', 'wb') as handle:
    pickle.dump(tot_list_test, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [103]:
with open('../Data/test_data_ACSF_id.pickle', 'wb') as handle:
    pickle.dump(test_id, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [20]:
def save_type(prefix,type_list):
    for i,type_ in enumerate(type_list):
        with open(prefix+'_type_'+str(i)+'.pickle', 'wb') as handle:
            pickle.dump(type_, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [101]:
save_type('../Data/train_data_ACSF',type_list_train)
save_type('../Data/val_data_ACSF',type_list_val)
save_type('../Data/test_data_ACSF',type_list_test)

In [105]:
save_type('../Data/test_data_ACSF_id',test_id_type)

numpy -> torch

In [2]:
with open('../Data/train_data_ACSF.pickle', 'rb') as handle:
    train_data = pickle.load(handle)
with open('../Data/val_data_ACSF.pickle', 'rb') as handle:
    val_data = pickle.load(handle)
with open('../Data/test_data_ACSF.pickle', 'rb') as handle:
    test_data = pickle.load(handle)

In [3]:
def load_type(prefix):
    data = []
    for i in range(8):
        with open(prefix+'_type_'+str(i)+'.pickle', 'rb') as handle:
            data.append(pickle.load(handle))
    return data

In [5]:
train_data_type = load_type('../Data/train_data_ACSF')
val_data_type = load_type('../Data/val_data_ACSF')
test_data_type = load_type('../Data/test_data_ACSF')

In [10]:
# convert numpy array to torch array
import torch
train_data = [{k:torch.tensor(i[k]) for k in i.keys()} for i in train_data]
val_data = [{k:torch.tensor(i[k]) for k in i.keys()} for i in val_data]
test_data = [{k:torch.tensor(i[k]) for k in i.keys()} for i in test_data]

In [11]:
def numpy2torch(type_list):
    out = []
    for type_ in type_list:
        out.append([{k:torch.tensor(i[k]) for k in i.keys()} for i in type_])
    return out

In [12]:
train_data_type = numpy2torch(train_data_type)
val_data_type = numpy2torch(val_data_type)
test_data_type = numpy2torch(test_data_type)

In [13]:
with open('../Data/train_data_ACSF.pickle', 'wb') as handle:
    pickle.dump(train_data, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('../Data/val_data_ACSF.pickle', 'wb') as handle:
    pickle.dump(val_data, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('../Data/test_data_ACSF.pickle', 'wb') as handle:
    pickle.dump(test_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [21]:
save_type('../Data/train_data_ACSF',train_data_type)
save_type('../Data/val_data_ACSF',val_data_type)
save_type('../Data/test_data_ACSF',test_data_type)